In [1]:
import pandas as pd
import numpy as np
import datetime

import feather
pd.options.display.max_columns = 999

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
temp = train.groupby(['user_id'])["listen_type"].sum().reset_index()
temp.columns = ['user_id', 'n']
train = pd.merge(train, temp, how='left', on='user_id')

In [4]:
train.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened,n
0,25471,1480597215,222606,41774,12,20040704,1,0,223,0,0,9241,55164,29,0,84
1,25571,1480544735,250467,43941,0,20060301,2,1,171,0,0,16547,55830,30,1,1
2,16,1479563953,305197,48078,1,20140714,2,1,149,1,1,7665,2704,29,1,88
3,7,1480152098,900502,71521,0,20001030,0,0,240,0,1,1580,938,30,0,294
4,7,1478368974,542335,71718,0,20080215,0,0,150,0,1,1812,2939,24,1,192


In [5]:
train = train.sort_values(["user_id", "ts_listen"])

In [6]:
def preprocess(df):
    df_temp = df.copy()
    for col in ['genre_id', 'media_id', 'album_id', 'artist_id']:
        temp = df_temp[col].value_counts().reset_index()
        temp.columns = [col, "n_{}".format(col)]
        temp["{}_new".format(col)] = temp.apply(lambda x: int(x[col]) if x["n_{}".format(col)] > 100 else -1, axis=1)
        df_temp = pd.merge(df_temp, temp[[col, "{}_new".format(col)]], how='left',on=[col])
    return df_temp

In [7]:
def prepare_train(df, fold):
    print("Fold {}".format(fold))
    df_temp = preprocess(df).copy()
    temp = df[(df["listen_type"] == 1) & (df["n"] >= fold)].groupby(['user_id']).apply(lambda x: x.iloc[-fold])["ts_listen"].reset_index()
    temp.columns = ["user_id", "ts_listen_last"]
    df_temp = pd.merge(df_temp, temp, how='left', on='user_id')
    df_temp = df_temp[df_temp['ts_listen'] <= df_temp['ts_listen_last']]
    feather.write_dataframe(df_temp[df_temp["n"] >= fold].groupby(["user_id"]).apply(lambda x: x.iloc[-1]), "folds/test_{}.feather".format(i))
    feather.write_dataframe(df_temp[df_temp["n"] >= fold].groupby(["user_id"]).apply(lambda x: x.iloc[:-1]), "folds/train_{}.feather".format(i))

In [8]:
for i in range(1,2):
    prepare_train(train, i)

Fold 1


In [9]:
for i in range(1,2):
    df = feather.read_dataframe("folds/train_{}.feather".format(i))
    print(i, df.shape[0], len(df.user_id.unique()))

1 6423310 15714


In [10]:
df.columns

Index(['genre_id', 'ts_listen', 'media_id', 'album_id', 'context_type',
       'release_date', 'platform_name', 'platform_family', 'media_duration',
       'listen_type', 'user_gender', 'user_id', 'artist_id', 'user_age',
       'is_listened', 'n', 'genre_id_new', 'media_id_new', 'album_id_new',
       'artist_id_new', 'ts_listen_last'],
      dtype='object')